In [1]:
# Installment
from urllib.request import urlopen
from bs4 import BeautifulSoup


In [2]:
def ScrapDataItem(url):
    html = urlopen(url).read()
    s = BeautifulSoup(html, "html.parser")
    # print(s.prettify())
    return s

def PrintTupleData(tuple):
    for data in tuple:
        print(data, end=" || ")
    print()

In [3]:
def ScrapProductHrefAndID(url):
    tempSoup = ScrapDataItem(url)
    allProducts = tempSoup.findAll("div", "dne-itemtile dne-itemtile-large")

    hrefArr = []
    for i in allProducts:
        listingID = i['data-listing-id']
        href = i.find("a")['href']
        hrefArr.append((href,listingID))
    return hrefArr

# Usage example
# test = ScrapProductHref("https://www.ebay.com/globaldeals/tech/laptops-netbooks")

In [41]:
def ScrapProductPage(url, type, listingID):    
    tempSoup = ScrapDataItem(url)

    name = tempSoup.find("h1", "x-item-title__mainTitle").find("span", class_="ux-textspans ux-textspans--BOLD").get_text()
    condition = tempSoup.find("div", class_="x-item-condition-value").find("span", class_="ux-textspans").get_text()

    availability = tempSoup.find("div", "d-quantity__availability")
    if (availability is not None):
        availability = availability.findAll("span", "ux-textspans")
        available = availability[0].get_text() # [:-9] #removing "available"
        if (len(availability) > 1): # if there is sold data
            sold = availability[2].get_text()[:-4] # removing "sold"
        else :
            sold = None
    else :
        available = None; sold = None
    
    paymentString = ""
    payment = tempSoup.find("div", "vim d-payments-minview").find("div", "ux-labels-values__values col-9")
    payment = payment.findAll("span",role="img")
    for i in payment:
        if (len(paymentString) == 0):
            paymentString = i['title']
        else:
            paymentString = paymentString + "," + i['title']

    product_price_original = tempSoup.find("div", "x-price-primary").find("span", "ux-textspans").get_text() # [4:] #removing "US $"
    product_price_idr = tempSoup.find("div", "x-price-approx").find("span", "ux-textspans ux-textspans--SECONDARY ux-textspans--BOLD").get_text() # [3:] #removing "IDR"
    shipping_price_original = tempSoup.find("div", "ux-labels-values__values-content").find("span", "ux-textspans ux-textspans--BOLD")
    if (shipping_price_original is not None):
        shipping_price_original = shipping_price_original.get_text() # [4:]  # remocing "US $"
    shipping_location = tempSoup.find("div", "ux-labels-values__values-content").find("span", "ux-textspans ux-textspans--SECONDARY")
    if (shipping_location is not None):
        shipping_location = shipping_location.get_text()[12:] # removing "located in: "
    return_info = tempSoup.find("div", "ux-layout-section ux-layout-section--returns").findAll("span", "ux-textspans")[1].get_text()
    seller = tempSoup.find("div", "d-stores-info-categories__container__info__section__title").find("span", "ux-textspans ux-textspans--BOLD").get_text()

    tuple = (listingID, name, type, available, sold, product_price_original, product_price_idr, shipping_price_original, shipping_location, return_info, seller, condition, paymentString)
    return tuple


In [42]:
prodsArray = [
    # ( categories, url )
    ( "Laptops & Netbooks" , "https://www.ebay.com/globaldeals/tech/laptops-netbooks"),
    ( "Cameras & Photo" , "https://www.ebay.com/globaldeals/tech/cameras-photo"),
    ( "TV, Video & Home Audio" , "https://www.ebay.com/globaldeals/tech/tv-video-home-audio" ),
    ( "iPads, Tablets & eReaders", "https://www.ebay.com/globaldeals/tech/ipads-tablets-ereaders"),
    ( "Video Games & Consoles", "https://www.ebay.com/globaldeals/tech/video-games-consoles"),
    ( "Phone Cases & Accessories", "https://www.ebay.com/globaldeals/tech/phone-cases-accessories"),
    ( "Vehicle Electronics & GPS", "https://www.ebay.com/globaldeals/tech/vehicle-electronics-gps"),
    ( "Memory Drives & Storage", "https://www.ebay.com/globaldeals/tech/memory-drives-storage"),
    ( "Printers & Printer Supplies", "https://www.ebay.com/globaldeals/tech/printers-printer-supplies"),
    ( "Headphones & Portable Audio", "https://www.ebay.com/globaldeals/tech/headphones-portable-audio"),
    ( "Computer Accessories", "https://www.ebay.com/globaldeals/tech/computer-accessories"),
    ( "Other Consumer Electronics", "https://www.ebay.com/globaldeals/tech/more-consumer-electronics")
]

In [43]:
# Scrap Every Data, make it insert it into a big array
dataArr = []
count = 0

for i in prodsArray:
    hrefArr = ScrapProductHrefAndID(i[1])
    for j in hrefArr:
        tempTuple = ScrapProductPage(j[0], i[0], j[1])
        if (count % 50 == 0):
            print(tempTuple)
        dataArr.append(tempTuple)
        count += 1

('333563280818', 'CHUWI Laptop HeroBook GemiBook CoreBook Pro/Plus/X Intel Core Laptop Windows', 'Laptops & Netbooks', '2 available', '32 ', 'EUR 239.99', 'IDR4,026,996.38', 'Free', None, '30 days returns', 'CHUWI Official Store（chuwi2018）', 'New', 'PayPal,Apple Pay,Google Pay,Visa,Master Card,American Express')
('204160038683', 'TV Wall Mount Wall Holder LCD LED TV 32-65/70" Tilt Black DE', 'TV, Video & Home Audio', 'More than 10 available', '74 ', 'EUR 14.50', 'IDR243,307.84', None, 'Euskirchen, Germany', '30 days returns', 'delightmark', 'New', 'PayPal,Apple Pay,Google Pay,Visa,Master Card,American Express')
('225553995775', 'Nintendo Switch Console 32GB - Device Only - Refurbished Good', 'Video Games & Consoles', 'Limited quantity available', '1 ', 'GBP 159.99', 'IDR3,132,580.32', 'GBP 56.95 ', 'Cannock, England, United Kingdom', '30 days returns', 'Stock Must Go', 'Good - Refurbished', 'PayPal,Apple Pay,Google Pay,Visa,Master Card,American Express')
('313292798496', 'Netac 120GB 2

# **DATAFRAME** (PANDAS AND NUMPY)

In [45]:
import numpy as np
import pandas as pd


# Making into data frame
numpyArr = np.array(dataArr)
df = pd.DataFrame(numpyArr)
df.columns = ["listing_id", "title", "type", "available", "sold", "price_original", "price_idr", "shipping_price_original", "shipping_location", "return", "seller", "condition", "payments"]
df

,listing_id,title,type,available,sold,price_original,price_idr,shipping_price_original,shipping_location,return,seller,condition,payments
0,333563280818,CHUWI Laptop HeroBook GemiBook CoreBook Pro/Pl...,Laptops & Netbooks,2 available,32,EUR 239.99,"IDR4,026,996.38",Free,None,30 days returns,CHUWI Official Store（chuwi2018）,New,"PayPal,Apple Pay,Google Pay,Visa,Master Card,A..."
1,333996205343,"CHUWI HeroBook Pro 14.1"" Windows 10 Laptop Int...",Laptops & Netbooks,Last One,14,EUR 199.99,"IDR3,355,802.35",Free,None,30 days returns,CHUWI Official Store（chuwi2018）,New,"PayPal,Apple Pay,Google Pay,Visa,Master Card,A..."
2,295671743764,"HP Omen 17.3"" Gaming Notebook QHD 165Hz i9-139...",Laptops & Netbooks,Limited quantity available,92,"US $2,499.99","IDR37,313,283.58",None,"Atlanta, Georgia, United States",30 days returns,Antonline,New,"PayPal,Apple Pay,Google Pay,Visa,Master Card,A..."
3,285185433173,"Lenovo ThinkPad P1 Gen 5 15.6"" 2022 2K 2.3GHz ...",Laptops & Netbooks,Limited quantity available,5,"US $1,399.99","IDR20,895,373.13",US $168.15,"Sanford, Florida, United States",30 days returns,ItsWorthMore,Excellent - Refurbished,"PayPal,Apple Pay,Google Pay,Visa,Master Card,A..."
4,283560024468,"Dell Latitude Laptop Computer 14"" Windows 10 P...",Laptops & Netbooks,More than 10 available,968,US $198.40,"IDR2,961,194.03",None,"Jacksonville, Texas, United States",Seller does not accept returns,Discount Computer Depot,Very Good - Refurbished,"PayPal,Apple Pay,Google Pay,Visa,Master Card,A..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,313784767380,Smart Wireless WiFi Ring Doorbell Security Int...,Other Consumer Electronics,More than 10 available,949,US $33.99/ea,"IDR507,313.43",US $24.71,"La Puente, California, United States",30 days returns,Whizzohead,New,"PayPal,Apple Pay,Google Pay,Visa,Master Card,A..."
218,285164489538,BOSS Audio Systems UNI4RGB UTV Marine Stereo P...,Other Consumer Electronics,More than 10 available,1,US $400.99,"IDR5,984,925.37",US $55.12,"Oxnard, California, United States",30 days returns,SpeeceInc,New,"PayPal,Apple Pay,Google Pay,Visa,Master Card,A..."
219,283619457023,BOSS Audio Systems MR692B 6 x 9 Inch Marine St...,Other Consumer Electronics,More than 10 available,87,US $65.00,"IDR970,149.25",US $58.89,"Oxnard, California, United States",30 days returns,SpeeceInc,New,"PayPal,Apple Pay,Google Pay,Visa,Master Card,A..."
220,284347290788,BOSS Audio Systems MRWT69 6 x 9 ATV UTV Marine...,Other Consumer Electronics,More than 10 available,81,US $106.35,"IDR1,587,313.43",US $69.11,"Oxnard, California, United States",30 days returns,SpeeceInc,New,"PayPal,Apple Pay,Google Pay,Visa,Master Card,A..."


In [ ]:
import os
dir = os.getcwd()
df.to_json(dir[:-3]+"data\data.json", orient='index', indent=2 )